# 7.7. 稠密连接网络（DenseNet）

ResNet极大地改变了如何参数化深层网络中函数的观点。
*稠密连接网络* (DenseNet） :cite:`Huang.Liu.Van-Der-Maaten.ea.2017` 在某种程度上是 ResNet 的逻辑扩展。让我们先从数学上了解一下。


## 7.7.1. 从ResNet到DenseNet

回想一下任意函数的泰勒展开式（Taylor expansion），它把这个函数分解成越来越高阶的项。在 $x$ 接近 0 时，

$$f(x) = f(0) + f'(0) x + \frac{f''(0)}{2!}  x^2 + \frac{f'''(0)}{3!}  x^3 + \ldots.$$

同样，ResNet 将函数展开为

$$f(\mathbf{x}) = \mathbf{x} + g(\mathbf{x}).$$

也就是说，ResNet 将 $f$ 分解为两部分：一个简单的线性项和一个更复杂的非线性项。
那么再向前拓展一步，如果我们想将 $f$ 拓展成超过两部分的信息呢？
一种方案便是 DenseNet。

![ResNet（左）与 DenseNet（右）在跨层连接上的主要区别：使用相加和使用连结。](../img/densenet-block.svg)
:label:`fig_densenet_block`

如 :numref:`fig_densenet_block` 所示，ResNet 和 DenseNet 的关键区别在于，DenseNet 输出是*连接*（用图中的 $[,]$ 表示）而不是如 ResNet 的简单相加。
因此，在应用越来越复杂的函数序列后，我们执行从 $\mathbf{x}$ 到其展开式的映射：

$$\mathbf{x} \to \left[
\mathbf{x},
f_1(\mathbf{x}),
f_2([\mathbf{x}, f_1(\mathbf{x})]), f_3([\mathbf{x}, f_1(\mathbf{x}), f_2([\mathbf{x}, f_1(\mathbf{x})])]), \ldots\right].$$

最后，将这些展开式结合到多层感知机中，再次减少特征的数量。
实现起来非常简单：我们不需要添加术语，而是将它们连接起来。
DenseNet 这个名字由变量之间的“稠密连接”而得来，最后一层与之前的所有层紧密相连。
稠密连接如 :numref:`fig_densenet` 所示。

![稠密连接。](../img/densenet.svg)
:label:`fig_densenet`

稠密网络主要由 2 部分构成： *稠密块*（dense block）和 *过渡层* （transition layer）。
前者定义如何连接输入和输出，而后者则控制通道数量，使其不会太复杂。


## (**7.7.2. 稠密块体**)

DenseNet 使用了 ResNet 改良版的“批量归一化、激活和卷积”结构（参见 :numref:`sec_resnet` 中的练习）。
我们首先实现一下这个结构。


In [23]:
import paddle
import paddle.nn as nn
import numpy as np

paddle.disable_static()

class BNConv(nn.Layer):
    def __init__(self, num_channels, num_filters):
        super(BNConv, self).__init__()
        model = [
            nn.BatchNorm2D(num_channels),
            nn.ReLU(),
            nn.Conv2D(num_channels, num_filters, 3, stride=1, padding=1)
        ]
        self.model = nn.Sequential(*model)
    def forward(self, X):
        return self.model(X)

一个*稠密块*由多个卷积块组成，每个卷积块使用相同数量的输出信道。
然而，在前向传播中，我们将每个卷积块的输入和输出在通道维上连结。


In [24]:
class DenseBlock(nn.Layer):
    def __init__(self, num_channels, num_layers, growth_rate):
        super(DenseBlock, self).__init__()
        self.dense_blocks = []
        for i in range(num_layers):
            block = self.add_sublayer(str(i), BNConv(num_channels + i * growth_rate, growth_rate))
            self.dense_blocks.append(block)
    def forward(self, X):
        for block in self.dense_blocks:
            X = paddle.concat([X, block(X)], axis=1)
        return X

在下面的例子中，我们[**定义一个**]有 2 个输出通道数为 10 的 (**`DenseBlock`**)。
使用通道数为 3 的输入时，我们会得到通道数为 $3+2\times 10=23$ 的输出。
卷积块的通道数控制了输出通道数相对于输入通道数的增长，因此也被称为*增长率*（growth rate）。


In [25]:
with paddle.fluid.dygraph.guard(paddle.fluid.cpu_places()[0]):
    blk = DenseBlock(3, 2, 10)
    X = paddle.to_tensor(np.random.uniform(-1., 1., [4, 3, 8, 8]).astype('float32'))
    Y = blk(X)
    print(Y.shape)

[4, 23, 8, 8]


## [**7.7.3. 过渡层**]

由于每个稠密块都会带来通道数的增加，使用过多则会过于复杂化模型。
而过渡层可以用来控制模型复杂度。
它通过 $1\times 1$ 卷积层来减小通道数，并使用步幅为 2 的平均池化层减半高和宽，从而进一步降低模型复杂度。


In [26]:
class TransitionLayer(nn.Layer):
    def __init__(self, num_channels, num_filters):
        super(TransitionLayer, self).__init__()
        model = [
            nn.BatchNorm2D(num_channels),
            nn.ReLU(),
            nn.Conv2D(num_channels, num_filters, 1, stride=1),
            nn.MaxPool2D(kernel_size=2, stride=2)
        ]
        self.model = nn.Sequential(*model)
    def forward(self, X):
        return self.model(X)

对上一个例子中稠密块的输出[**使用**]通道数为 10 的[**过渡层**]。
此时输出的通道数减为 10，高和宽均减半。


In [27]:
with paddle.fluid.dygraph.guard(paddle.fluid.cpu_places()[0]):
    blk = TransitionLayer(23, 10)
    Y = blk(Y)
    print(Y.shape)

[4, 10, 4, 4]


## [**7.7.4. DenseNet模型**]

我们来构造 DenseNet 模型。DenseNet 首先使用同 ResNet 一样的单卷积层和最大池化层。


In [29]:
model = [
            nn.Conv2D(1, 64, 7, stride=2, padding=3),
            nn.BatchNorm2D(64),
            nn.ReLU(),
            nn.MaxPool2D(kernel_size=3, stride=2, padding=1)
        ]

接下来，类似于 ResNet 使用的 4 个残差块，DenseNet 使用的是 4 个稠密块。
与 ResNet 类似，我们可以设置每个稠密块使用多少个卷积层。
这里我们设成 4，从而与 :numref:`sec_resnet` 的 ResNet-18 保持一致。
稠密块里的卷积层通道数（即增长率）设为 32，所以每个稠密块将增加 128 个通道。

在每个模块之间，ResNet 通过步幅为 2 的残差块减小高和宽，DenseNet 则使用过渡层来减半高和宽，并减半通道数。


In [31]:
num_channels, growth_rate = 64, 32  # num_channels为当前的通道数
# num_convs_in_dense_blocks = [4, 4, 4, 4]
num_convs_in_dense_blocks = [4, 4]
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    model += [DenseBlock(num_channels, num_convs, growth_rate)]
    # 上一个稠密块的输出通道数
    num_channels += num_convs * growth_rate
    # 在稠密块之间加入通道数减半的过渡层
    if i != len(num_convs_in_dense_blocks) - 1:
        model += [TransitionLayer(num_channels, num_channels // 2)]
        num_channels //= 2

与 ResNet 类似，最后接上全局池化层和全连接层来输出结果。


In [34]:
class DenseNet(nn.Layer):
    def __init__(self, num_classes=10):
        super(DenseNet, self).__init__()

        # DenseNet首先使用同ResNet一样的单卷积层和最大池化层。
        model = [
            nn.Conv2D(1, 64, 7, stride=2, padding=3),
            nn.BatchNorm2D(64),
            nn.ReLU(),
            nn.MaxPool2D(kernel_size=3, stride=2, padding=1)
        ]

        # 类似于ResNet接下来使用的4个残差块，DenseNet使用的是4个稠密块。
        # 同ResNet一样，我们可以设置每个稠密块使用多少个卷积层。
        # 这里我们设成4，从而与上一节的ResNet-18保持一致。
        # 稠密块里的卷积层通道数（即增长率）设为32，所以每个稠密块将增加128个通道。
        # ResNet里通过步幅为2的残差块在每个模块之间减小高和宽。这里我们则使用过渡层来减半高和宽，并减半通道数。
        num_channels, growth_rate = 64, 32  # num_channels为当前的通道数
        num_convs_in_dense_blocks = [4, 4, 4, 4]
        for i, num_convs in enumerate(num_convs_in_dense_blocks):
            model += [DenseBlock(num_channels, num_convs, growth_rate)]
            # 上一个稠密块的输出通道数
            num_channels += num_convs * growth_rate
            # 在稠密块之间加入通道数减半的过渡层
            if i != len(num_convs_in_dense_blocks) - 1:
                model += [TransitionLayer(num_channels, num_channels // 2)]
                num_channels //= 2

        # 同ResNet一样，最后接上全局池化层和全连接层来输出。
        model += [
            nn.AdaptiveAvgPool2D(output_size=1),
            nn.Flatten(start_axis=1, stop_axis=-1),
            nn.Linear(num_channels, num_classes),
        ]
        self.model = nn.Sequential(*model)
    def forward(self, X):
        Y = self.model(X)
        return Y

dn = DenseNet(10)
X = paddle.to_tensor(np.random.uniform(-1., 1., [4, 1, 96, 96]).astype('float32'))
Y = dn(X)
print(Y.shape)

[4, 10]


## [**7.7.5. 训练模型**]

由于这里使用了比较深的网络，本节里我们将输入高和宽从 224 降到 96 来简化计算。


In [36]:
import paddle
import paddle.vision.transforms as T
from paddle.vision.datasets import FashionMNIST

# 数据集处理
transform = T.Compose([
    T.Resize(96),
    T.Transpose(),
    T.Normalize([127.5], [127.5]),
])
train_dataset = FashionMNIST(mode='train', transform=transform)
val_dataset = FashionMNIST(mode='test', transform=transform)
# 模型定义
model = paddle.Model(DenseNet(10))
# 设置训练模型所需的optimizer, loss, metric
model.prepare(
    paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters()),
    paddle.nn.CrossEntropyLoss(),
    paddle.metric.Accuracy(topk=(1, 5)))
# 启动训练、评估
model.fit(train_dataset, val_dataset, epochs=2, batch_size=64, log_freq=100)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/2
step 100/938 - loss: 0.3307 - acc_top1: 0.7370 - acc_top5: 0.9759 - 33ms/step
step 200/938 - loss: 0.3445 - acc_top1: 0.7860 - acc_top5: 0.9861 - 33ms/step
step 300/938 - loss: 0.3763 - acc_top1: 0.8104 - acc_top5: 0.9895 - 34ms/step
step 400/938 - loss: 0.3491 - acc_top1: 0.8245 - acc_top5: 0.9917 - 34ms/step
step 500/938 - loss: 0.3414 - acc_top1: 0.8347 - acc_top5: 0.9931 - 34ms/step
step 600/938 - loss: 0.1988 - acc_top1: 0.8429 - acc_top5: 0.9940 - 34ms/step
step 700/938 - loss: 0.2294 - acc_top1: 0.8494 - acc_top5: 0.9946 - 34ms/step
step 800/938 - loss: 0.4946 - acc_top1: 0.8552 - acc_top5: 0.9952 - 34ms/step
step 900/938 - loss: 0.1301 - acc_top1: 0.8590 - acc_top5: 0.9956 - 34ms/step
step 938/938 - loss: 0.3350 - acc_top1: 0.8600 - acc_top5: 0.9957 - 34ms/step
Eval begin...
step 100/157 - loss: 0.1935 - acc_top1: 0.8892 - acc_top5: 0.9973 - 20ms/step
step 157

## 7.7.6. 小结

* 在跨层连接上，不同于 ResNet 中将输入与输出相加，稠密连接网络（DenseNet）在通道维上连结输入与输出。
* DenseNet 的主要构建模块是稠密块和过渡层。
* 在构建 DenseNet 时，我们需要通过添加过渡层来控制网络的维数，从而再次减少信道的数量。


## 7.7.7. 练习

1. 为什么我们在过渡层使用平均池化层而不是最大池化层？
1. DenseNet 的优点之一是其模型参数比 ResNet 小。为什么呢？
1. DenseNet 一个诟病的问题是内存或显存消耗过多。
    1. 真的是这样吗？可以把输入形状换成 $224 \times 224$ ，来看看实际的显存消耗。
    1. 你能想出另一种方法来减少显存消耗吗？你需要如何改变框架？
1. 实现 DenseNet 论文 :cite:`Huang.Liu.Van-Der-Maaten.ea.2017` 表 1 所示的不同 DenseNet 版本。
1. 应用DenseNet的思想设计一个基于多层感知机的模型。将其应用于 :numref:`sec_kaggle_house` 中的房价预测任务。


[Discussions](https://discuss.d2l.ai/t/1880)
